# CIFAR-10 Classification with Hyperparameter Tuning using Ray Tune

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt
import numpy as np

import ray
from ray import tune
from ray.tune import Tuner, TuneConfig
from ray.tune.schedulers import ASHAScheduler
from ray.air import session


In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
torch.backends.cudnn.benchmark = True
print("Using device:", device)


Using device: cuda


In [3]:
transform_train = transforms.Compose([
    transforms.RandomHorizontalFlip(),
    transforms.RandomCrop(32, padding=4),
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465),(0.2023, 0.1994, 0.2010)),
])

transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465),(0.2023, 0.1994, 0.2010)),
])

train_val_set = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform_train)
test_set = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform_test)

num_train = 49000
num_val = 1000
train_set, val_set = torch.utils.data.random_split(train_val_set, [num_train, len(train_val_set)-num_train])

train_loader = DataLoader(train_set, batch_size=128, shuffle=True, num_workers=4, pin_memory=True)
val_loader = DataLoader(val_set, batch_size=128, shuffle=False, num_workers=4, pin_memory=True)
test_loader = DataLoader(test_set, batch_size=128, shuffle=False, num_workers=4, pin_memory=True)


In [4]:
from train_cnn import train_cnn_model

In [5]:
tuner_cnn = tune.Tuner(
    tune.with_resources(
        tune.with_parameters(train_cnn_model, train_loader=train_loader, val_loader=val_loader),
        resources={"cpu": 20, "gpu": 1}
    ),
    param_space={
    "lr": tune.choice([
        1e-2,   # Fast learning, might overshoot
        5e-3,
        1e-3,   # Commonly works well
        5e-4,
        1e-4,   # Slower but safer convergence
    ]),
    "reg": tune.choice([
        1e-5,    # Very light regularization
        1e-4,    # Commonly used
        5e-4,
        1e-3,
        5e-3,    # Strong regularization (may underfit)
    ]),
        "epochs": 20
    },
    tune_config=tune.TuneConfig(
        num_samples=50,
        metric="val_accuracy",
        mode="max",
        scheduler=ASHAScheduler(),
        max_concurrent_trials=20
    )
)

results_cnn = tuner_cnn.fit()
best_cnn = results_cnn.get_best_result(metric="val_accuracy", mode="max")
print("Best CNN config:", best_cnn.config)
print("Best CNN accuracy:", best_cnn.metrics["val_accuracy"])


(train_cnn_model pid=237427) /home/sraja/miniconda3/envs/pytorch/lib/python3.12/site-packages/ray/train/_internal/session.py:772: RayDeprecationWarning: `ray.train.report` should be switched to `ray.tune.report` when running in a function passed to Ray Tune. This will be an error in the future. See this issue for more context: https://github.com/ray-project/ray/issues/49454
(train_cnn_model pid=237427)   _log_deprecation_warning(
(train_cnn_model pid=238182) /home/sraja/miniconda3/envs/pytorch/lib/python3.12/site-packages/ray/train/_internal/session.py:772: RayDeprecationWarning: `ray.train.report` should be switched to `ray.tune.report` when running in a function passed to Ray Tune. This will be an error in the future. See this issue for more context: https://github.com/ray-project/ray/issues/49454
(train_cnn_model pid=238182)   _log_deprecation_warning(
(train_cnn_model pid=238778) /home/sraja/miniconda3/envs/pytorch/lib/python3.12/site-packages/ray/train/_internal/session.py:772: Ra

Best CNN config: {'lr': 0.005, 'reg': 0.0001, 'epochs': 20}
Best CNN accuracy: 0.721
